In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
print(f"AWS bucket: {bucket}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

AWS bucket: sagemaker-us-east-1-348052051973
AWS account: 348052051973
AWS region: us-east-1


In [16]:
train_s3 = {
    'model': sagemaker.inputs.TrainingInput(s3_data='s3://nlp-348052051973/model/llama-7b-hf/'),
    #'model': sagemaker.inputs.TrainingInput(s3_data='s3://nlp-sagemakers/models/llama-hf-13b', content_type='application/x-sagemaker-model'),
    'data': sagemaker.inputs.TrainingInput(s3_data='s3://nlp-348052051973/data')
}

In [17]:
from sagemaker.pytorch import PyTorch

In [18]:
from sagemaker import get_execution_role
role = get_execution_role()

In [19]:
image_uri = '348052051973.dkr.ecr.us-east-1.amazonaws.com/alpaca:latest'
job_name = 'nlp-sft-persona-7b'

In [20]:
metric_definitions=[
        {'Name': 'loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
        {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

In [30]:
pytorch_estimator = PyTorch(base_job_name=job_name,
                            source_dir="./stanford_alpaca",
                            entry_point="train-llama7B.sh",  #  the entry point that launches the training script with options
                            role=role,
                            image_uri=image_uri,
                            instance_type='ml.p4d.24xlarge',
                            instance_count=1,
                            metric_definitions=metric_definitions,
                            # hyperparameters=hyperparameters,
                            #distribution=distribution
                           )

In [31]:
pytorch_estimator.fit(train_s3)
# pytorch_estimator.fit()

INFO:sagemaker:Creating training-job with name: nlp-sft-persona-7b-2023-04-10-12-58-28-129


2023-04-10 12:58:33 Starting - Starting the training job...
2023-04-10 12:59:09 Starting - Preparing the instances for training............
2023-04-10 13:00:59 Downloading - Downloading input data......
2023-04-10 13:01:49 Training - Downloading the training image...............
2023-04-10 13:04:31 Training - Training image download completed. Training in progress.....2023-04-10 13:05:20,367 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-10 13:05:20,467 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-10 13:05:20,575 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-10 13:05:20,583 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "data": "/opt/ml/input/data/data",
        "model": "/opt/ml/input/data/model"
    },
    "current_host": "algo-1"

UnexpectedStatusException: Error for Training job nlp-sft-persona-7b-2023-04-10-12-58-28-129: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 127
ErrorMessage ""
Command "/bin/sh -c ./train-llama7B.sh ", exit code: 1

In [16]:
sm = boto3.client("sagemaker")


def stop_training_job(name):
    status = sm.describe_training_job(TrainingJobName=name)["TrainingJobStatus"]
    if status == "InProgress":
        sm.stop_training_job(TrainingJobName=name)

print(pytorch_estimator.latest_training_job.name)
stop_training_job(pytorch_estimator.latest_training_job.name)

nlp-alapca-125m-2023-03-24-04-03-18-624
